In [1]:
pip install transformers trl datasets accelerate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, OPTForCausalLM, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig, RewardTrainer, RewardConfig, DataCollatorForCompletionOnlyLM
from datasets import load_dataset, Dataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset = load_dataset("nvidia/HelpSteer2")

train_dataset = dataset['train']
validation_dataset = dataset['validation']

README.md:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.jsonl.gz:   0%|          | 0.00/582k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1038 [00:00<?, ? examples/s]

В датасете представлены колонки с оценками пользователей в 5 категориях: 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity. Для каждого prompt имеется 2 responses модели. Для выбора chosen и rejected авторы предлагают следующую формулу (Appendix H [HelpSteer2: Open-source dataset for training top-performing reward models](https://arxiv.org/pdf/2406.08673)):

0.65 $\cdot$ Helpfulness + 0.8 $\cdot$ Correctness + 0.45 $\cdot$ Coherence + 0.55 $\cdot$ Complexity - 0.4 $\cdot$ Verbosity

In [5]:
df_train = pd.DataFrame(train_dataset)
df_val = pd.DataFrame(validation_dataset)

df_train['score_1'] = df_train.apply(lambda row: 0.65 * row['helpfulness'] + 0.8 * row['correctness'] + 0.45 * row['coherence'] + 0.55 * row['complexity'] - 0.4 * row['verbosity'], axis=1)
df_val['score_1'] = df_val.apply(lambda row: 0.65 * row['helpfulness'] + 0.8 * row['correctness'] + 0.45 * row['coherence'] + 0.55 * row['complexity'] - 0.4 * row['verbosity'], axis=1)

df_train = df_train.drop(columns=['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'])
df_val = df_val.drop(columns=['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'])

In [5]:
model_name = 'facebook/opt-350m'
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

In [7]:
def preprocessing_for_sft(examples):

    return tokenizer(examples['prompt'], examples['response'], padding="max_length", truncation=True, max_length=512)

In [8]:
def best_responses(df):
    responses_dict = {col: [] for col in df.columns}

    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i + 1]
            score1 = row1['score_1']
            score2 = row2['score_1']

            if score1 >= score2:
                chosen_row = row1
            else:
                chosen_row = row2

            for col in df.columns:
                responses_dict[col].append(chosen_row[col])

    data_best =  Dataset.from_dict(responses_dict)

    return data_best

Отбираем лучшие ответы:

In [9]:
train_data = best_responses(df_train)
val_data = best_responses(df_val)

train_data = train_data.remove_columns(['score_1'])
val_data = val_data.remove_columns(['score_1'])

In [10]:
train_data_sft = train_data.map(preprocessing_for_sft, batched=True)
val_data_sft = val_data.map(preprocessing_for_sft, batched=True)

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

**Рассмотрим 2 варианта.**

**1)Первый вариант**: SFTTrainer по дефолту использует DataCollatorForLanguageModeling ([here](https://github.com/huggingface/trl/blob/974b0d380f12c357b70265c5f2dd2c8cb39a6a3e/trl/trainer/sft_trainer.py#L337-L338)), который не отделяет prompt от response при вычислении CrossEntropyLoss. Обучим модель с помощью него.

In [11]:
training_args = TrainingArguments(
    output_dir='./results/sft',
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch'
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data_sft,
    eval_dataset=val_data_sft,
    tokenizer=tokenizer,
    data_collator=None,
    max_seq_length=512
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [ ]:
trainer.train()

Скачаем уже обученную: [here](https://drive.google.com/drive/folders/1D2CxM0_1_GXgg7LXO4JRKuS7d_5q5hFY?usp=drive_link).

In [15]:
model_sft_name = 'SFT_model'
model_sft = AutoModelForCausalLM.from_pretrained(model_sft_name)
model_sft.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

Обучим для нее Reward Model. Подготовим датасет.

In [16]:
def preprocessing_for_chosen(examples):

    return tokenizer(examples['chosen'], padding='max_length', truncation=True, max_length=512)

def preprocessing_for_rejected(examples):

    return tokenizer(examples['rejected'], padding='max_length', truncation=True, max_length=512)

def pairs_for_rm(df):
    responses_dict = {col: [] for col in ['chosen', 'rejected']}

    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i + 1]
            score1 = row1['score_1']
            score2 = row2['score_1']

            if score1 >= score2:
                responses_dict['chosen'].append(row1['response'])
                responses_dict['rejected'].append(row2['response'])
            else:
                responses_dict['chosen'].append(row2['response'])
                responses_dict['rejected'].append(row1['response'])

    data_pairs =  Dataset.from_dict(responses_dict)

    return data_pairs

In [17]:
train_data_rm = pairs_for_rm(df_train)
val_data_rm = pairs_for_rm(df_val)

In [18]:
train_data_rm = train_data_rm.map(preprocessing_for_chosen, batched=True)

train_data_rm = train_data_rm.remove_columns(['chosen'])
train_data_rm = train_data_rm.rename_columns({'input_ids': 'input_ids_chosen'})
train_data_rm = train_data_rm.rename_columns({'attention_mask': 'attention_mask_chosen'})

train_data_rm = train_data_rm.map(preprocessing_for_rejected, batched=True)

train_data_rm = train_data_rm.remove_columns(['rejected'])
train_data_rm = train_data_rm.rename_columns({'input_ids': 'input_ids_rejected'})
train_data_rm = train_data_rm.rename_columns({'attention_mask': 'attention_mask_rejected'})

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

In [19]:
val_data_rm = val_data_rm.map(preprocessing_for_chosen, batched=True)

val_data_rm = val_data_rm.remove_columns(['chosen'])
val_data_rm = val_data_rm.rename_columns({'input_ids': 'input_ids_chosen'})
val_data_rm = val_data_rm.rename_columns({'attention_mask': 'attention_mask_chosen'})

val_data_rm = val_data_rm.map(preprocessing_for_rejected, batched=True)

val_data_rm = val_data_rm.remove_columns(['rejected'])
val_data_rm = val_data_rm.rename_columns({'input_ids': 'input_ids_rejected'})
val_data_rm = val_data_rm.rename_columns({'attention_mask': 'attention_mask_rejected'})

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

Reward Model из библиотеки trl в качестве reward при обучении использует среднее значение logits, полученных для всех токенов [here](https://github.com/huggingface/trl/blob/974b0d380f12c357b70265c5f2dd2c8cb39a6a3e/trl/trainer/reward_trainer.py#L243-L245). При оценке модели она усредняет logits для каждого токена в отдельности [here](https://github.com/huggingface/trl/blob/974b0d380f12c357b70265c5f2dd2c8cb39a6a3e/trl/trainer/reward_trainer.py#L279), что вызывает ошибку с размерностью:

In [20]:
training_args = RewardConfig(
    output_dir='./results/reward',
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer = RewardTrainer(
    model=model_sft,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args,
    eval_dataset=val_data_rm,
)

In [21]:
trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:330: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3697.)
  logits = torch.stack(logits).mean(dim=2).softmax(dim=0).T


TypeError: type list doesn't define __round__ method

**1.1** Рассмотрим Reward Model, которая будет усреднять logits только для последнего слоя. Обучим ее в течении 2-х эпох:

In [31]:
import reward_model_last_logits
from reward_model_last_logits import Reward_Model_ll

In [32]:
training_args_ll = RewardConfig(
    output_dir='./results/reward_ll',
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer_ll = Reward_Model_ll(
    model=model_sft,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_ll,
    eval_dataset=val_data_rm,
)

In [ ]:
trainer_ll.train()

Скачаем уже обученную модель: [here](https://drive.google.com/drive/folders/1EGf7hu0FKySXjJVbRUp2vTxxrK5_Ud89?usp=drive_link).

In [34]:
reward_model_ll_name = 'Reward_Model_LL'
reward_model_ll = AutoModelForCausalLM.from_pretrained(reward_model_ll_name)
reward_model_ll.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

In [35]:
trainer_ll = Reward_Model_ll(
    model=reward_model_ll,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_ll,
    eval_dataset=val_data_rm,
)

In [36]:
trainer_ll.evaluate()

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_values_avg   ┃ rejected_values_avg ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 0.41864527938342966 │ 0.42497109826589596 │
└─────────────────────┴─────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                  ┃ rejected_text                                ┃ logits            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ </s>In the context of NoSQL databases,       │ </s>In SQL, master-slave replication is a    │ [0.5264, 0.5177]  │
│ master-slave replication refers to a         │ technique used to create a copy of a         │                   │
│ configuration where a single master node     │ database on a separate server. The master    │                   │
│ writes data, and one or more slave nodes     │ server is the primary server that contains   │                   │
│ read data from the master and replicate it   │ the original data, while the slave server is │                   │
│ to provide read scalability. The master node │ the secondary server that contains a copy of │                   │
│ is responsible for accepting write requests  │ the data. The master server sends updates to │                   │
│ and updating its own data, while the slave   │ the slave server, which then applies them to │                   │
│ nodes are responsible for replicating the    │ its own database.                            │                   │
│ data from the master and serving read        │                                              │                   │
│ requests.                                    │ Here's how master-slave replication works:   │                   │
│                                              │                                              │                   │
│ In this configuration, the master node is    │ 1. The master server sends a stream of       │                   │
│ the only node that can make changes to the   │ updates to the slave server, which includes  │                   │
│ data, while the slave nodes can only read    │ information about changes made to the        │                   │
│ the data and replicate it. This ensures that │ database on the master server.               │                   │
│ the data is consistent across all nodes and  │                                              │                   │
│ that there is no data loss in case of a      │ 2. The slave server receives the updates and │                   │
│ failure of a single node.                    │ applies them to its own database, creating a │                   │
│                                              │ copy of the master server's database.        │                   │
│ Master-slave replication in NoSQL databases  │                                              │                   │
│ can be implemented using various             │ 3. The slave server can also send updates    │                   │
│ technologies, such as Apache Cassandra,      │ back to the master server, which can be used │                   │
│ MongoDB, and Apache HBase. In each of these  │ to keep the two databases in sync. This is   │                   │
│ databases, the master-slave replication      │ known as two-way replication.                │                   │
│ configuration can be set up using specific   │                                              │                   │
│ configuration options and parameters.        │ 4. If the master server fails, the slave     │                   │
│                                              │ server can take over as the new master       │                   │
│ It's worth noting that master-slave          │ server, ensuring that the database remains   │                   │
│ replication is not a failover solution, as   │ available.                                   │                   │
│ the failure of the master node will result   │                                              │                   │
│ in the loss of data until the node is        │ Master-

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 0.7035278677940369,
 'eval_accuracy': 0.48747591522157996,
 'eval_runtime': 97.1148,
 'eval_samples_per_second': 5.344,
 'eval_steps_per_second': 0.34}

Accuracy модели на отложенной выборке оказывается случайным угадыванием. Рассмотрим другой вариант.



**1.2** Заменим последний линейный слой модели lm_head на nn.Linear(512, 1), который будет генерировать 1 значение - reward для данного response. Обучим модель в течение 2-х эпох:

In [44]:
import reward_model_one_reward
from reward_model_one_reward import Reward_Model_OR

In [12]:
class ModelWithReward(OPTForCausalLM):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.reward = nn.Linear(512, 1)

    def score(self, x):

        return self.reward(x)

In [39]:
del (model, model_sft, reward_model_ll)
torch.cuda.empty_cache()

In [40]:
model_sft_name = 'SFT_model'
model_one_reward = ModelWithReward.from_pretrained(model_sft_name)
model_one_reward.to(device)

Some weights of ModelWithReward were not initialized from the model checkpoint at SFT_model and are newly initialized: ['reward.bias', 'reward.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModelWithReward(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_feature

In [41]:
del model_one_reward.lm_head

In [42]:
model_one_reward

ModelWithReward(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_feature

In [45]:
training_args_or = RewardConfig(
    output_dir='./results/reward_one',
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer_or = Reward_Model_OR(
    model=model_one_reward,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_or,
    eval_dataset=val_data_rm,
)

In [ ]:
trainer_or.train()

Скачаем уже обученную модель на двух чекпоинтах:



Чекпоинт после 1 эпохи: [here](https://drive.google.com/drive/folders/1F2Dji35Eubnse4UYWUF89XBEqSkPFSLh?usp=sharing).



Чекпоинт после 2 эпохи: [here]().

In [48]:
model_or_1_name = 'Reward_Model_OR_1'
model_or_1 = ModelWithReward.from_pretrained(model_or_1_name)
model_or_1.to(device)
del model_or_1.lm_head

In [49]:
training_args_or = RewardConfig(
    output_dir='./results/reward_ll',
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer_or_1 = Reward_Model_OR(
    model=model_or_1,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_or,
    eval_dataset=val_data_rm,
)

In [50]:
trainer_or_1.evaluate()

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_values_avg   ┃ rejected_values_avg ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ -0.8822533718689788 │ -1.0702402697495184 │
└─────────────────────┴─────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                 ┃ rejected_text                                ┃ logits             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ </s>In the context of NoSQL databases,      │ </s>In SQL, master-slave replication is a    │ [-0.8735, -1.4721] │
│ master-slave replication refers to a        │ technique used to create a copy of a         │                    │
│ configuration where a single master node    │ database on a separate server. The master    │                    │
│ writes data, and one or more slave nodes    │ server is the primary server that contains   │                    │
│ read data from the master and replicate it  │ the original data, while the slave server is │                    │
│ to provide read scalability. The master     │ the secondary server that contains a copy of │                    │
│ node is responsible for accepting write     │ the data. The master server sends updates to │                    │
│ requests and updating its own data, while   │ the slave server, which then applies them to │                    │
│ the slave nodes are responsible for         │ its own database.                            │                    │
│ replicating the data from the master and    │                                              │                    │
│ serving read requests.                      │ Here's how master-slave replication works:   │                    │
│                                             │                                              │                    │
│ In this configuration, the master node is   │ 1. The master server sends a stream of       │                    │
│ the only node that can make changes to the  │ updates to the slave server, which includes  │                    │
│ data, while the slave nodes can only read   │ information about changes made to the        │                    │
│ the data and replicate it. This ensures     │ database on the master server.               │                    │
│ that the data is consistent across all      │                                              │                    │
│ nodes and that there is no data loss in     │ 2. The slave server receives the updates and │                    │
│ case of a failure of a single node.         │ applies them to its own database, creating a │                    │
│                                             │ copy of the master server's database.        │                    │
│ Master-slave replication in NoSQL databases │                                              │                    │
│ can be implemented using various            │ 3. The slave server can also send updates    │                    │
│ technologies, such as Apache Cassandra,     │ back to the master server, which can be used │                    │
│ MongoDB, and Apache HBase. In each of these │ to keep the two databases in sync. This is   │                    │
│ databases, the master-slave replication     │ known as two-way replication.                │                    │
│ configuration can be set up using specific  │                                              │                    │
│ configuration options and parameters.       │ 4. If the master server fails, the slave     │                    │
│                                             │ server can take over as the new master       │                    │
│ It's worth noting that master-slave         │ server, ensuring that the database remains   │                    │
│ replication is not a failover solution, as  │ available.                                   │                    │
│ the failure of the master node will result  │                                              │                    │
│ in the loss of data until the node is       │ Master-s

{'eval_loss': 0.6538317799568176,
 'eval_accuracy': 0.5992292870905588,
 'eval_runtime': 100.4462,
 'eval_samples_per_second': 5.167,
 'eval_steps_per_second': 0.329}

In [51]:
model_or_2_name = 'Reward_Model_OR_2'
model_or_2 = ModelWithReward.from_pretrained(model_or_2_name)
model_or_2.to(device)

del model_or_2.lm_head

In [52]:
trainer_or_2 = Reward_Model_OR(
    model=model_or_2,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_or,
    eval_dataset=val_data_rm,
)

In [53]:
trainer_or_2.evaluate()

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_values_avg    ┃ rejected_values_avg ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 0.059246435452793816 │ -0.3353747591522158 │
└──────────────────────┴─────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                 ┃ rejected_text                                ┃ logits             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ </s>In the context of NoSQL databases,      │ </s>In SQL, master-slave replication is a    │ [1.9585, 0.2561]   │
│ master-slave replication refers to a        │ technique used to create a copy of a         │                    │
│ configuration where a single master node    │ database on a separate server. The master    │                    │
│ writes data, and one or more slave nodes    │ server is the primary server that contains   │                    │
│ read data from the master and replicate it  │ the original data, while the slave server is │                    │
│ to provide read scalability. The master     │ the secondary server that contains a copy of │                    │
│ node is responsible for accepting write     │ the data. The master server sends updates to │                    │
│ requests and updating its own data, while   │ the slave server, which then applies them to │                    │
│ the slave nodes are responsible for         │ its own database.                            │                    │
│ replicating the data from the master and    │                                              │                    │
│ serving read requests.                      │ Here's how master-slave replication works:   │                    │
│                                             │                                              │                    │
│ In this configuration, the master node is   │ 1. The master server sends a stream of       │                    │
│ the only node that can make changes to the  │ updates to the slave server, which includes  │                    │
│ data, while the slave nodes can only read   │ information about changes made to the        │                    │
│ the data and replicate it. This ensures     │ database on the master server.               │                    │
│ that the data is consistent across all      │                                              │                    │
│ nodes and that there is no data loss in     │ 2. The slave server receives the updates and │                    │
│ case of a failure of a single node.         │ applies them to its own database, creating a │                    │
│                                             │ copy of the master server's database.        │                    │
│ Master-slave replication in NoSQL databases │                                              │                    │
│ can be implemented using various            │ 3. The slave server can also send updates    │                    │
│ technologies, such as Apache Cassandra,     │ back to the master server, which can be used │                    │
│ MongoDB, and Apache HBase. In each of these │ to keep the two databases in sync. This is   │                    │
│ databases, the master-slave replication     │ known as two-way replication.                │                    │
│ configuration can be set up using specific  │                                              │                    │
│ configuration options and parameters.       │ 4. If the master server fails, the slave     │                    │
│                                             │ server can take over as the new master       │                    │
│ It's worth noting that master-slave         │ server, ensuring that the database remains   │                    │
│ replication is not a failover solution, as  │ available.                                   │                    │
│ the failure of the master node will result  │                                              │                    │
│ in the loss of data until the node is       │ Master-s

{'eval_loss': 0.9171338677406311,
 'eval_accuracy': 0.5645472061657033,
 'eval_runtime': 101.1294,
 'eval_samples_per_second': 5.132,
 'eval_steps_per_second': 0.326}

Модель начинает переобучаться. Возникло предположение, что, возможно, невысокое значение accuracy связано со слишком сложной формулой для вычисления score, разделяющего chosen и rejected.

**2) Второй вариант:** для более тонкого обучения используем DataCollatorForCompletionOnlyLM. Он вычисляет CrossEntropyLoss только на основе response. Разделять chosen и rejected будем просто на основе 'helpfulness'.

In [54]:
del (model_one_reward, model_or_1, model_or_2)
torch.cuda.empty_cache()

In [55]:
df_train = pd.DataFrame(train_dataset)
df_val = pd.DataFrame(validation_dataset)

df_train = df_train.drop(columns=['correctness', 'coherence', 'complexity', 'verbosity'])
df_val = df_val.drop(columns=['correctness', 'coherence', 'complexity', 'verbosity'])

In [56]:
def best_helpfulness(df):
    responses_dict = {col: [] for col in df.columns}

    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i + 1]
            score1 = row1['helpfulness']
            score2 = row2['helpfulness']

            if score1 >= score2:
                chosen_row = row1
            else:
                chosen_row = row2

            for col in df.columns:
                responses_dict[col].append(chosen_row[col])

    data_best =  Dataset.from_dict(responses_dict)

    return data_best

In [57]:
train_data = best_helpfulness(df_train)
val_data = best_helpfulness(df_val)

train_data = train_data.remove_columns(['helpfulness'])
val_data = val_data.remove_columns(['helpfulness'])

In [58]:
model_name = "facebook/opt-350m"

model_sft_help = AutoModelForCausalLM.from_pretrained(model_name)
model_sft_help.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [59]:
def formatting_prompts_func(example):
    output_texts = []

    for i in range(len(example['prompt'])):
        text = f"### Question: {example['prompt'][i]}\n ### Answer: {example['response'][i]}"
        output_texts.append(text)

    return output_texts

response_template = ' ### Answer:'
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./sft_help',
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

trainer = SFTTrainer(
    model=model_sft_help,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=512
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Скачаем уже обученную: [here](https://drive.google.com/drive/folders/1HcrMbd0U_ZEBPWZgUnuxOhxuBYyzNM8X?usp=sharing).

In [69]:
model_name_help = 'SFT_help'
model_sft_help = ModelWithReward.from_pretrained(model_name_help)
model_sft_help.to(device)
del model_sft_help.lm_head

Some weights of ModelWithReward were not initialized from the model checkpoint at SFT_help and are newly initialized: ['reward.bias', 'reward.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Обучим для нее в течение 1 эпохи Reward Model, которая выдает 1 reward на последнем слое.

In [73]:
def preprocessing_for_chosen(examples):

    return tokenizer(examples['chosen'], padding='max_length', truncation=True, max_length=512)

def preprocessing_for_rejected(examples):

    return tokenizer(examples['rejected'], padding='max_length', truncation=True, max_length=512)

def pairs_for_rm_help(df):
    responses_dict = {col: [] for col in ['chosen', 'rejected']}

    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i + 1]
            score1 = row1['helpfulness']
            score2 = row2['helpfulness']

            if score1 >= score2:
                responses_dict['chosen'].append(row1['response'])
                responses_dict['rejected'].append(row2['response'])

            else:
                responses_dict['chosen'].append(row2['response'])
                responses_dict['rejected'].append(row1['response'])

    data_pairs =  Dataset.from_dict(responses_dict)

    return data_pairs

In [74]:
df_train = pd.DataFrame(train_dataset)
df_val = pd.DataFrame(validation_dataset)

train_data_rm = pairs_for_rm_help(df_train)
val_data_rm = pairs_for_rm_help(df_val)

In [75]:
train_data_rm = train_data_rm.map(preprocessing_for_chosen, batched=True)

train_data_rm = train_data_rm.remove_columns(['chosen'])
train_data_rm = train_data_rm.rename_columns({'input_ids': 'input_ids_chosen'})
train_data_rm = train_data_rm.rename_columns({'attention_mask': 'attention_mask_chosen'})

train_data_rm = train_data_rm.map(preprocessing_for_rejected, batched=True)

train_data_rm = train_data_rm.remove_columns(['rejected'])
train_data_rm = train_data_rm.rename_columns({'input_ids': 'input_ids_rejected'})
train_data_rm = train_data_rm.rename_columns({'attention_mask': 'attention_mask_rejected'})

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

In [76]:
val_data_rm = val_data_rm.map(preprocessing_for_chosen, batched=True)

val_data_rm = val_data_rm.remove_columns(['chosen'])
val_data_rm = val_data_rm.rename_columns({'input_ids': 'input_ids_chosen'})
val_data_rm = val_data_rm.rename_columns({'attention_mask': 'attention_mask_chosen'})

val_data_rm = val_data_rm.map(preprocessing_for_rejected, batched=True)

val_data_rm = val_data_rm.remove_columns(['rejected'])
val_data_rm = val_data_rm.rename_columns({'input_ids': 'input_ids_rejected'})
val_data_rm = val_data_rm.rename_columns({'attention_mask': 'attention_mask_rejected'})

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [77]:
training_args_or = RewardConfig(
    output_dir='./results/reward_help',
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer_or = Reward_Model_OR(
    model=model_sft_help,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_or,
    eval_dataset=val_data_rm,
)

In [ ]:
trainer_or.train()

Скачаем уже обученную модель: [here](https://drive.google.com/drive/folders/1IRUcY9swWPu6MCmvrxd8uTKZQyXTL5Bi?usp=sharing).

In [14]:
model_or_name = 'SFT_help_RM'
model_or = ModelWithReward.from_pretrained(model_or_name)
model_or.to(device)

del model_or.lm_head

In [79]:
trainer_or = Reward_Model_OR(
    model=model_or,
    train_dataset=train_data_rm,
    processing_class=tokenizer,
    args=training_args_or,
    eval_dataset=val_data_rm,
)

In [80]:
trainer_or.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_values_avg    ┃ rejected_values_avg ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ -0.18526531791907516 │ -0.3642564547206166 │
└──────────────────────┴─────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                 ┃ rejected_text                                ┃ logits             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ </s>In the context of NoSQL databases,      │ </s>In SQL, master-slave replication is a    │ [0.2786, -0.1357]  │
│ master-slave replication refers to a        │ technique used to create a copy of a         │                    │
│ configuration where a single master node    │ database on a separate server. The master    │                    │
│ writes data, and one or more slave nodes    │ server is the primary server that contains   │                    │
│ read data from the master and replicate it  │ the original data, while the slave server is │                    │
│ to provide read scalability. The master     │ the secondary server that contains a copy of │                    │
│ node is responsible for accepting write     │ the data. The master server sends updates to │                    │
│ requests and updating its own data, while   │ the slave server, which then applies them to │                    │
│ the slave nodes are responsible for         │ its own database.                            │                    │
│ replicating the data from the master and    │                                              │                    │
│ serving read requests.                      │ Here's how master-slave replication works:   │                    │
│                                             │                                              │                    │
│ In this configuration, the master node is   │ 1. The master server sends a stream of       │                    │
│ the only node that can make changes to the  │ updates to the slave server, which includes  │                    │
│ data, while the slave nodes can only read   │ information about changes made to the        │                    │
│ the data and replicate it. This ensures     │ database on the master server.               │                    │
│ that the data is consistent across all      │                                              │                    │
│ nodes and that there is no data loss in     │ 2. The slave server receives the updates and │                    │
│ case of a failure of a single node.         │ applies them to its own database, creating a │                    │
│                                             │ copy of the master server's database.        │                    │
│ Master-slave replication in NoSQL databases │                                              │                    │
│ can be implemented using various            │ 3. The slave server can also send updates    │                    │
│ technologies, such as Apache Cassandra,     │ back to the master server, which can be used │                    │
│ MongoDB, and Apache HBase. In each of these │ to keep the two databases in sync. This is   │                    │
│ databases, the master-slave replication     │ known as two-way replication.                │                    │
│ configuration can be set up using specific  │                                              │                    │
│ configuration options and parameters.       │ 4. If the master server fails, the slave     │                    │
│                                             │ server can take over as the new master       │                    │
│ It's worth noting that master-slave         │ server, ensuring that the database remains   │                    │
│ replication is not a failover solution, as  │ available.                                   │                    │
│ the failure of the master node will result  │                                              │                    │
│ in the loss of data until the node is       │ Master-s

{'eval_loss': 0.6543089747428894,
 'eval_accuracy': 0.6165703275529865,
 'eval_runtime': 100.8425,
 'eval_samples_per_second': 5.147,
 'eval_steps_per_second': 0.327}

Accuracy немного возросла! Посмотрим на средний reward на генерациях SFT модели.

In [82]:
from transformers import GenerationConfig
from trl.trainer.utils import batch_generation
import numpy as np

del (trainer_or, trainer, model_sft_help)
torch.cuda.empty_cache()

In [13]:
model_name_help = 'SFT_help'
model_sft_help = AutoModelForCausalLM.from_pretrained(model_name_help)
model_sft_help.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

In [87]:
def get_score(model, inputs):
    logits = model.model(input_ids=inputs)['last_hidden_state']
    rewards = model.reward(logits[:, -1, :])

    return rewards

score_dict = {'score': [] }

In [15]:
def preprocessing_for_val_sft(examples):

    return tokenizer(examples['prompt'], padding='max_length', truncation=True, max_length=512)

In [16]:
def pairs_for_val_sft(df):
    responses_dict = {'prompt': []}

    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            row = df.iloc[i + 1]
            responses_dict['prompt'].append(row['prompt'])

    data =  Dataset.from_dict(responses_dict)

    return data

In [89]:
df_val = pd.DataFrame(validation_dataset)

val_data = pairs_for_val_sft(df_val)
val_data = val_data.map(preprocessing_for_val_sft, batched=True)

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [97]:
args = GenerationConfig(max_new_tokens=100)

for i in range(len(val_data['input_ids'])):
    if i % 100 == 0:
        print(f'Сгенерированно {i} ответов')

    with torch.no_grad():
        input_tensor = torch.LongTensor([val_data['input_ids'][i]]).to(device)
        new_response = batch_generation(
                                         model_sft_help,
                                         input_tensor,
                                         local_rollout_forward_batch_size=1,
                                         pad_token_id=tokenizer.pad_token_id,
                                         generation_config=args
        )[0][:, 512:]

        torch.cuda.empty_cache()

        score = get_score(model_or, new_response).cpu()
        score_dict['score'].append(score)

        del input_tensor, new_response, score
        torch.cuda.empty_cache()

Сгенерированно 0 ответов
Сгенерированно 100 ответов
Сгенерированно 200 ответов
Сгенерированно 300 ответов
Сгенерированно 400 ответов
Сгенерированно 500 ответов


Оценим средний reward генераций SFT модели до RLHF.

In [98]:
np.mean(score_dict['score'])

-0.9487341

**Теперь обучим алгоритм Reinforce.**

In [17]:
import reinforce
from reinforce import Reinforce_Model

In [18]:
df_train = pd.DataFrame(train_dataset)
df_val = pd.DataFrame(validation_dataset)

In [19]:
train_data_rein = pairs_for_val_sft(df_train)
val_data_rein = pairs_for_val_sft(df_val)

train_data_rein = train_data_rein.map(preprocessing_for_val_sft, batched=True)
val_data_rein = val_data_rein.map(preprocessing_for_val_sft, batched=True)

train_data_rein = train_data_rein.remove_columns(['prompt'])
val_data_rein = val_data_rein.remove_columns(['prompt'])

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [20]:
import copy

model_sft_help.to('cpu')
model_copy = copy.deepcopy(model_sft_help)
model_sft_help.to(device)
model_copy.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

In [21]:
from trl.trainer import RLOOConfig

training_args = RLOOConfig(
    output_dir='./results/reinforce',
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer = Reinforce_Model(
    policy=model_sft_help,
    ref_policy=model_copy,
    reward_model=model_or,
    train_dataset=train_data_rein,
    processing_class=tokenizer,
    config=training_args,
    eval_dataset=val_data_rein,
)

world_size 1
local_batch_size 4
args.gradient_accumulation_steps 1
num_total_batches 5081
local_dataloader_batch_size 4


In [22]:
trainer.train()

===training policy===


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ explain master slave replication nsql       │                                             │ -1.0585137605667114 │
│                                             │                                             │                     │
│                                             │ In NSQL, a master slave is a single-purpose │                     │
│                                             │ object that can be accessed and managed by  │                     │
│                                             │ a single user. The master slave is          │                     │
│                                             │ responsible for managing the data and       │                     │
│                                             │ ensuring that the data is accessed and      │                     │
│                                             │ managed correctly.                          │                     │
│                                             │                                             │                     │
│                                             │ In general, a master slave is a type of     │                     │
│                                             │ object that can be accessed and managed by  │                     │
│                                             │ a single user. The master slave is          │                     │
│                                             │ responsible for creating and maintaining    │                     │
│                                             │ the data, ensuring that it is accessed and  │                     │
│                                             │ managed correctly.                          │                     │
│                                             │                                             │                     │
│                                             │ In NSQL, a master slave is a type of object │                     │
│                                             │ that can be accessed and managed by a       │                     │
│                                             │ single user. The master slave is            │                     │
│                                             │ responsible for creating and maintaining    │                     │
│                                             │ the data,                                   │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ I am studying for a clinical informatics    │                                             │ -0.3231334984302521 │
│ board exam that will have multiple-choice   │                                             │                     │
│ questions. I will be asking you 86 of these │ Note: This table is a general overview of   │                     │
│ questions. When I give you each question    │ the questions you will face, but it is not  │                     │
│ and its answer options, please create a     │ a exhaustive list. Any specific questions   │                     │
│ table that includes the following           │ or questions that you may encounter will be │                     │
│ information: the question number, the       │ considered as part of the exam. The         │                     │
│ question stem, the letter of the correct    │ specific questions you will encounter will  │                     │
│ answer, an brief explanation of what you    │ depend on the level of knowledge and        │                     │
│ think the correct answer is. Additionally,  │ understa

KeyboardInterrupt: 

Обученная модель не сохранилась. Впрочем, она и не обучилась с такой Reward Model:(

In [23]:
trainer.generate_completions()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                             ┃ score                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ explain master slave replication nsql       │ :                                          │ -0.23118919134140015 │
│                                             │                                            │                      │
│                                             │ Master slave replication (MSR) is a        │                      │
│                                             │ technique used to replicate data from one  │                      │
│                                             │ machine to another. It involves the use of │                      │
│                                             │ a master or slave cluster to replicate     │                      │
│                                             │ data across different machines.            │                      │
│                                             │                                            │                      │
│                                             │ MSR is typically used for data replication │                      │
│                                             │ across multiple machines, such as in a     │                      │
│                                             │ database or a shared memory. It works by   │                      │
│                                             │ using a master or slave cluster to         │                      │
│                                             │ replicate data across different machines,  │                      │
│                                             │ and then using a master or slave cluster   │                      │
│                                             │ to replicate data across different         │                      │
│                                             │ machines.                                  │                      │
│                                             │                                            │                      │
│                                             │ MSR is a powerful tool for data            │                      │
│                                             │ replication across multiple machines, as   │                      │
│                                             │ it allows organizations to efficiently     │                      │
│                                             │ replicate data across different types of   │                      │
│                                             │ data sources, such as databases            │                      │
├─────────────────────────────────────────────┼────────────────────────────────────────────┼──────────────────────┤
│ I am studying for a clinical informatics    │                                            │ -0.9726682901382446  │
│ board exam that will have multiple-choice   │                                            │                      │
│ questions. I will be asking you 86 of these │ Question Number:                           │                      │
│ questions. When I give you each question    │                                            │                      │
│ and its answer options, please create a     │ 1. What is the name of the person or group │                      │
│ table that includes the following           │ of people who make up the city or region   │                      │
│ information: the question number, the       │ where you live?                            │                      │
│ question stem, the letter of the correct    │                                            │                      │
│ answer, an brief explanation of what you    │ 2. What 